In [12]:
import qiime2
import os
os.getcwd()
#change working dir to above
os.chdir('/home/rtsantos3/bioinfo_pipelines/picrust2-2.4.2/data/Kiara-Qiime2/')

In [6]:
!qiime tools import   \
--type 'SampleData[PairedEndSequencesWithQuality]'   \
--input-path 'read-files/sample-manifest.tsv'   \
--output-path paired-end-demux.qza   \
--input-format PairedEndFastqManifestPhred33V2 \



Imported read-files/sample-manifest.tsv as PairedEndFastqManifestPhred33V2 to paired-end-demux.qza


In [8]:
#Denoising Step

!qiime dada2 denoise-single \
    --i-demultiplexed-seqs paired-end-demux.qza \
    --p-trunc-len 140 \
    --p-n-threads 2 \
    --output-dir dada2 --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/q2-SingleLanePerSampleSingleEndFastqDirFmt-y0l7b_jb /tmp/tmp8_8_mqns/output.tsv.biom /tmp/tmp8_8_mqns/track.tsv /tmp/tmp8_8_mqns 140 0 2.0 2 Inf independent consensus 1.0 2 1000000 NULL 16

R version 4.0.5 (2021-03-31) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering ....
2) Learning Error Rates
39056920 total bases in 278978 reads from 4 samples will be used for learning the error rates.
3) Denoise samples ....
4) Remove chimeras (method = consensus)
5) Report read numbers through the pipeline
6) Write output
Saved FeatureTable[Frequency] to: dada2/table.qza
Saved FeatureData[Sequence] to: dada2/representative_sequences.qza
Saved SampleData[DADA2Stats] to: dada2/denoising_stats

In [20]:
!qiime feature-classifier fit-classifier-naive-bayes \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --o-classifier "TaxonomicClassifier.qza" \
        --output-dir "../training-sets/Classifier" \
        --verbose 

/home/rtsantos3/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/q2_feature_classifier/classifier.py:102: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.24.1. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


In [24]:
!qiime feature-classifier classify-consensus-vsearch \
        --i-query "../dada2/representative_sequences.qza" \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --output-dir "../taxonomy_vsearch" 

Saved FeatureData[Taxonomy] to: ../taxonomy_vsearch/classification.qza


In [ ]:
!qiime feature-classifier classify-sklearn --help \
        --i-reads "../dada2/representative_sequences.qza" \
        --i-classifier "../training-sets/"

In [26]:
!qiime tools export \
    --input-path "../taxonomy_vsearch/classification.qza" \
    --output-path "../taxonomy_vsearch/"

Exported ../taxonomy_vsearch/classification.qza as TSVTaxonomyDirectoryFormat to directory ../taxonomy_vsearch/


In [9]:
!qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising-stats.qzv

Saved Visualization to: dada2/denoising-stats.qzv


In [13]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada2/representative_sequences.qza \
    --output-dir tree

Saved FeatureData[AlignedSequence] to: tree/alignment.qza
Saved FeatureData[AlignedSequence] to: tree/masked_alignment.qza
Saved Phylogeny[Unrooted] to: tree/tree.qza
Saved Phylogeny[Rooted] to: tree/rooted_tree.qza


In [14]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

Saved Visualization to: tree/empress.qzv


In [40]:
#Alpha Diversity stat

!qiime diversity core-metrics-phylogenetic \
    --i-table dada2/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 10000 \
    --m-metadata-file 'read-files/metadata.tsv' \
    --output-dir diversity

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

In [26]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file 'read-files/metadata.tsv' \
    --o-visualization diversity/alpha_groups.qzv

Saved Visualization to: diversity/alpha_groups.qzv


In [39]:
#Permanova via Adonis addon

!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file read-files/metadata.tsv \
    --p-formula "treatment" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

Saved Visualization to: diversity/permanova.qzv


In [26]:
#Running Picrust2 module on ASVs obtained as well as on the representative sequences

!qiime picrust2 full-pipeline\
    --i-table dada2/table.qza \
    --i-seq dada2/representative_sequences.qza\
    --output-dir q2-picrust2 \
    --p-placement-tool sepp \
    --p-threads 6 \
    --p-hsp-method pic \
    --p-max-nsti 2 \
    --verbose

Warning - 1 input sequences aligned poorly to reference sequences (--min_align option specified a minimum proportion of 0.8 aligning to reference sequences). These input sequences will not be placed and will be excluded from downstream steps.

This is the set of poorly aligned input sequences to be excluded: 7e879359d8a9e9b3c08b121533609d54





All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.

All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.


Saved FeatureTable[Frequency] to: q2-picrust2/ko_metagenome.qza
Saved FeatureTable[Frequency] to: q2-picrust2/ec_metagenome.qza
Saved FeatureTable[Frequency] to: q2-picrust2/pathway_abundance.qza


In [27]:
#Generate picrust2 feature table
!qiime feature-table summarize \
    --i-table q2-picrust2/pathway_abundance.qza \
    --o-visualization q2-picrust2/pathway_abundance.qzv 

Saved Visualization to: q2-picrust2/pathway_abundance.qzv


In [28]:
#Export pathway tables to biom format

!qiime tools export \
    --input-path q2-picrust2/pathway_abundance.qza \
    --output-path pathabun_exported 




Exported q2-picrust2/pathway_abundance.qza as BIOMV210DirFmt to directory pathabun_exported


In [7]:
#Export KO table to biom format

!qiime tools export \
    --input-path q2-picrust2/ko_metagenome.qza \
    --output-path ko_metagenome

!biom convert \
    -i ko_metagenome/ko-feature-table.biom \
    -o ko_metagenome/ko-feature-table.biom.tsv \
    --to-tsv

Exported q2-picrust2/ko_metagenome.qza as BIOMV210DirFmt to directory ko_metagenome


In [6]:
#Export EC table to biom format

!qiime tools export \
    --input-path q2-picrust2/ec_metagenome.qza \
    --output-path ec_metagenome

!biom convert \
    -i ec_metagenome/ec-feature-table.biom \
    -o ec_metagenome/ec-feature-table.biom.tsv \
    --to-tsv

Exported q2-picrust2/ec_metagenome.qza as BIOMV210DirFmt to directory ec_metagenome


In [29]:
!biom convert \
    -i pathabun_exported/feature-table.biom \
    -o pathabun_exported/feature-table.biom.tsv \
    --to-tsv

In [5]:
#Convert DADA2 output to fasta and tsv, respectively

!qiime tools export \
    --input-path dada2/table.qza \
    --output-path Stratified-pipeline/

!qiime tools export \
    --input-path dada2/representative_sequences.qza \
    --output-path Stratified-pipeline/rep_seqs.fasta


Exported dada2/table.qza as BIOMV210DirFmt to directory Stratified-pipeline/
Exported dada2/representative_sequences.qza as DNASequencesDirectoryFormat to directory Stratified-pipeline/rep_seqs.fasta
